### 1. Import relevant packages

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cmocean as cmo

# cartopy 
import cartopy.crs as ccrs       # ccrs contains information about projections
import cartopy    
import cartopy.feature as cfeature
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator)
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER

### 2. Construct the url where your data is located and define file names

In [2]:
base_url = 'https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2021/' 
year_day = str(360)


filename = 'A2021' + year_day + '.L3m_DAY_CHL_chlor_a_4km.nc' # change year/file of interest here

url = base_url + year_day + '/' + filename

In [3]:
url

'https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2021/360/A2021360.L3m_DAY_CHL_chlor_a_4km.nc'

### 3. Use a for loop to plot multiple days of chlorophyll data and save them as .png files

In [8]:
for day in range(120,130):
    year_day = str(day)

    filename = 'A2021' + year_day + '.L3m_DAY_CHL_chlor_a_4km.nc'
    time  = filename[1:5] + year_day
    url = base_url + year_day + '/' + filename
    modis = xr.open_dataset(url)
    subset = modis.where((modis.lon>=-173.5)&(modis.lon<-147)& (modis.lat>=51)&(modis.lat<73), drop=True)
    fig = plt.figure(figsize=(10,10))
    regional_extent = [-173.5, -147, 51, 73]
    ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=-160))
    ax.set_extent(regional_extent)


    ax.add_feature(cartopy.feature.LAND,edgecolor='black',zorder=1) 
    ax.add_feature(cartopy.feature.RIVERS,zorder=2)

    states = cfeature.NaturalEarthFeature(category='cultural',
                                          name='admin_1_states_provinces_lines',
                                          scale='50m',
                                          facecolor='none'
    )
    ax.add_feature(states, edgecolor='gray', zorder=3)

    gl = ax.gridlines(crs=ccrs.PlateCarree(),
                      draw_labels=True, 
                      dms=True, 
                      x_inline=False, 
                      y_inline=False
    )
    gl.xlocator = mticker.FixedLocator(np.arange(-180,-130,5))
    gl.ylocator = mticker.FixedLocator(np.arange(50,75,5))
    ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
    ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)

    # plots the chlorophyll files
    im = subset.chlor_a.plot.pcolormesh(x='lon',
                                        y='lat', 
                                        norm=colors.LogNorm(vmin=0.1,vmax=5),
                                        cmap='Spectral_r',
                                        transform=ccrs.PlateCarree(),
    )
    plt.title(time)
    print(year_day)
    figname = time + '.png'
    fig.savefig('figures/'+figname, bbox_inches='tight', dpi=150) # saves figures in a different folder
    plt.close()

120
121
122
123
124
125
126
127
128
129


### 4. Average time periods of interest
Averaging over a period of time can be useful in several scenarios. Today we are using it to compare El Nino and La Nina events in LIS. However, it can also be useful to see the general range of chlorophyll concentrations in a particular area, especially if that area has poor coverage via satellites. 

### 5. Optional: create a .gif for a period of time 
1. Open a terminal and go to the directory where your .png chlorophyll map files are. 
2. Copy and run the following line of code:

In [ ]:
convert -delay 50 -loop 0 *.png animation.gif
#Note: the number after delay determines how much time elapses between figures, and you can change the name of your .gif file. 

3. Open the .gif file in Jupyterlab to view it. 
4. Optional: go to https://cloudconvert.com/ to convert your .gif file into another type of video file, such as .mp4